In [ ]:
def calc_window(shape):
    """from enlib import enmap"""
    """Compute fourier-space window function. Like the other fourier-based
    functions in this module, equi-spaced pixels are assumed. Since the
    window function is separable, it is returned as an x and y part,
    such that window = wy[:,None]*wx[None,:]."""
    wy = np.sinc(np.fft.fftfreq(shape[-2]))
    wx = np.sinc(np.fft.fftfreq(shape[-1]))
    return wy, wx

def filter_map(m, apo, kx, ky, legacy_steve=True, unpixwin=True):
    k_r0 = enmap.fft(m)
    ly, lx = enmap.lmap(k_r0.shape, k_r0.wcs)
    kx_bool = (np.abs(lx) <= kx)
    ky_bool = (np.abs(ly) <= ky)
    
    if unpixwin:
        wy, wx = calc_window(k_r0.shape)
        k_r0 /= wy[:,None]
        k_r0 /= wx[None,:]
    
    if legacy_steve:
        cut_x_k = np.unique(lx[(np.abs(lx) <= kx )])
        cut_y_k = np.unique(ly[(np.abs(ly) <= ky )])
        # do not cut the most positive wavenumber
        print((cut_x_k[-1]))
        print((cut_y_k[-1]))
        kx_bool = kx_bool & ((lx) < (cut_x_k[-1]))
        ky_bool = ky_bool & ((ly) < (cut_y_k[-1]))
    
    k_r0[kx_bool] = 0.0
    k_r0[ky_bool] = 0.0
    r0_diy = enmap.ifft(k_r0)
    
    special_apo = apo.copy()
    special_apo[ np.isclose(special_apo, 0.0)] = np.inf
    
    result = r0_diy.real
    result[1:,1:] = result[:-1,:-1] / special_apo[1:, 1:]
    return (result) / special_apo
zack_maps = [
    filter_map(m * apo, apo, kx, ky)
    for m in boxed_maps_f]

In [ ]:
xmin, xmax = 3000, 3000+50
yval = 700

fig, axes = plt.subplots(1, 2, figsize=(10,4))
axes[0].plot( ((steve_maps[0] - zack_maps[0])[yval,1000:3200]), label='diff' )
axes[0].plot( ((steve_maps[0])[yval,1000:3200])/100, label='ps/100', alpha=0.2 )
axes[0].legend()

ymin, ymax = 700, 700+50
xval = 3000
axes[1].plot( (steve_maps[0] - zack_maps[0])[40:1400,xval] )
axes[1].plot( (steve_maps[0])[40:1400,xval] /100, alpha=0.2)

plt.tight_layout();